In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman2
from sklearn.datasets import make_friedman3
from sklearn.base import BaseEstimator, RegressorMixin
from skrvm import RVR
from sklearn.grid_search import GridSearchCV

In [2]:
def EM_re_estimation(self, n_iterations, likelihood=float('inf')):
        gamma = np.array([0.] * self.kernel_m)
        w_mp = np.matrix([[]])
        t_matrix = np.matrix(self.data_t)

        last = self.marginal_likelihood()
        cur = last
        print("FIRST LIKELIHOOD = ", last)
        iterations_number = 0
        log_proba = [cur]
        relevant_vectors_number = [self.kernel_m]

        # for k in range(n_iterations):
        while cur < likelihood and iterations_number < n_iterations:
            last = cur
            iterations_number += 1

            tmp_m = self.betta * (self.F.T * self.F) + np.matrix(np.diag(self.alpha))
            sigma = np.linalg.inv(self.betta * (self.F.T * self.F) + np.matrix(np.diag(self.alpha)))
            w_mp = sigma * self.F.T * (self.betta * t_matrix)
            for j in range(self.kernel_m):
                if np.abs(w_mp.A[j]) < self.weight_bound or np.abs(self.alpha[j]) > self.alpha_bound:
                    w_mp.A[j] = 0
                    self.alpha[j] = 1e200
                    gamma[j] = 0
                else:
                    gamma[j] = 1 - self.alpha[j] * sigma.A[j][j]
                    self.alpha[j] = 1 / (w_mp.A[j][0] ** 2 + sigma.A[j][j])

            self.betta = (self.N) / (np.linalg.norm(t_matrix - self.F * np.matrix(w_mp)) ** 2
                                     + sum(gamma) / self.betta)

            cur = self.marginal_likelihood()
#             print(cur - last, cur, iterations_number)
            
            log_proba.append(cur)
            relevant_vectors_number.append((np.abs(w_mp) > self.eps).sum())

        print(iterations_number, "\nLIKELIHOOD", last)
        self.w = w_mp

        return log_proba, relevant_vectors_number

In [38]:
from scipy.special import expit
from scipy.linalg import logm, expm
from sklearn.base import BaseEstimator
from sklearn.metrics.pairwise import rbf_kernel, linear_kernel, polynomial_kernel 

class RVMRegression(BaseEstimator, RegressorMixin):
    """RVM Regression class"""
    fitted = False
    alpha = None
    betta = None
    kernel = 'linear'
    data_x = None
    data_t = None
    alpha_bound = 10**12
    weight_bound = 10**-3
    number_of_iterations = 100
    F = None
    kernel_m = None
    D = 0
    w = None
    inv_a = None
    N = 0
    eps = 1e-3
    lokelihood_eps = 1e-4

    def __init__(self, algorithm='tipping', learning_rate=0.1, alpha=None, betta=None, 
                 kernel='rbf', coef0=0, gamma=1, degree=2, number_of_iterations=5000,
                 bias_used=True, betta_fixed=False, calc_prob=False):
        if alpha:
            self.alpha = alpha

        if betta:
            self.betta = betta

        self.algorithm = algorithm
        self.coef0 = coef0
        self.degree = degree
        self.gamma = gamma
        self.kernel = kernel
        self.number_of_iterations = number_of_iterations
        self.relevance_w = np.array([])
        self.relevance_x = np.array([])
        self.relevance_y = np.array([])
        self.learning_rate = learning_rate
        self.bias_used = bias_used
        self.betta_fixed = betta_fixed
        self.calc_prob = calc_prob

    def tipping_re_estimation(self, n_iterations):
        gamma = np.zeros(self.kernel_m)
        t = self.data_t.reshape((-1,))

        last = self.marginal_likelihood()
        cur = last
#         print("FIRST = ", last)
        iterations_number = 0
        log_proba = [cur]
        relevant_vectors_number = [self.kernel_m]


        for k in range(n_iterations):
            last = cur
            iterations_number += 1

            sigma = np.linalg.inv(self.betta * np.dot(self.F.T, self.F) + np.diag(self.alpha))
            w_mp = self.betta * np.dot(sigma, np.dot(self.F.T, t))
            
            alpha_old = self.alpha.copy()
            
            if self.algorithm == 'tipping':
                gamma = 1 - self.alpha * np.diag(sigma)
                self.alpha = gamma / (w_mp ** 2)

            elif self.algorithm == 'EM':
                self.alpha[j] = 1 / (w_mp ** 2 + np.diag(sigma))
            else:
                print('No such algorithm')
                
            
            if self.algorithm == 'tipping' and not self.betta_fixed:
                # ОПАСНО!!!!
                self.betta = (self.N - np.sum(gamma)) / (np.linalg.norm(t - np.dot(self.F, w_mp)) ** 2)
            elif self.algorithm == 'EM' and not self.betta_fixed:
                self.betta = (self.N) / (np.linalg.norm(t - np.dot(self.F, w_mp)) ** 2
                                         + sum(gamma) / self.betta)
            elif not self.betta_fixed:
                print('No such algorithm')
                
            
            rel_ind = self.alpha < self.alpha_bound
            
            self.alpha = self.alpha[rel_ind]
            gamma = gamma[rel_ind]
            w_mp = w_mp[rel_ind]
            self.F = self.F[:, rel_ind]
            sigma = sigma[np.ix_(rel_ind, rel_ind)]
            
            if self.bias_used:
                if not rel_ind[-1]:
                    self.bias_used = False
                self.relevance_x = self.relevance_x[rel_ind[:-1]]
            else:
                self.relevance_x = self.relevance_x[rel_ind]

            if self.calc_prob:
                cur = self.marginal_likelihood()
                log_proba.append(cur)
                print(cur, len(self.alpha))
#             relevant_vectors_number.append((np.abs(w_mp) > self.eps).sum())

        self.w = w_mp

        return log_proba, relevant_vectors_number

    def fit(self, x, t):
        if len(x) == 0:
            raise NameError("X array is empty!")
        if len(x) != len(t):
            raise NameError("Different len X and t")

        self.D = x.shape[1]
        self.N = x.shape[0]

        self.kernel_m = self.N + (1 if self.bias_used else 0)
                
        if self.kernel == '2D ex':
            self.kernel_m = len(x) + 4
        
        self.data_x = x.copy()
        self.data_t = t.copy()
        self.relevance_x = self.data_x

        if not self.alpha:
            self.alpha = 1
        
        self.alpha = np.ones(self.kernel_m) * self.alpha
        
        if not self.betta:
            self.betta = 1

        self.F = self._apply_kernel(self.data_x, self.data_x)
        
        log_prob, relevant_vectors_number = self.tipping_re_estimation(self.number_of_iterations)

#         self.rel_ind = np.abs(self.w) > self.weight_bound
        
#         self.relevance_w = self.w[self.rel_ind]
#         self.relevance_x = self.data_x[self.rel_ind, :]
#         self.relevance_y = self.data_t[self.rel_ind]

        return np.array(log_prob), np.array(relevant_vectors_number)

    
    def _apply_kernel(self, x, y):
        """Apply the selected kernel function to the data."""
        
        if self.kernel == 'linear':
            phi = linear_kernel(x, y)
        elif self.kernel == 'rbf':
            phi = rbf_kernel(x, y, self.gamma)
        elif self.kernel == 'poly':
            phi = polynomial_kernel(x, y, self.degree, self.gamma, self.coef0)
        elif callable(self.kernel):
            phi = self.kernel(x, y)
            if len(phi.shape) != 2:
                raise ValueError(
                    "Custom kernel function did not return 2D matrix"
                )
            if phi.shape[0] != x.shape[0]:
                raise ValueError(
                    "Custom kernel function did not return matrix with rows"
                    " equal to number of data points."""
                )
        else:
            raise ValueError("Kernel selection is invalid.")

        if self.bias_used:
            phi = np.append(phi, np.ones((phi.shape[0], 1)), axis=1)

        return phi


    def calc_sigma(self):
        a = np.matrix(np.diag(self.alpha))
        try:
            inv_a = np.linalg.inv(a)
        except np.linalg.LinAlgError:
            inv_a = np.linalg.pinv(a)

        self.inv_a = inv_a.copy()
        e = np.matrix(np.diag([1] * a.shape[0]))
        tmp_mat = np.linalg.pinv(e + self.betta * self.F * self.inv_a * self.F.T)
        return self.inv_a - self.inv_a * self.betta * self.F.T * tmp_mat * self.F * self.inv_a

    def predict(self, x):
        
        phi = self._apply_kernel(x, self.relevance_x)
        
        return np.dot(phi, self.w).reshape((-1,))
            
#     def score(self, data_x, t):
#         pred = self.predict(data_x)
        
#         u = ((pred.reshape((-1,)) - t.reshape((-1,))) ** 2).sum()
#         v = ((t.reshape((-1,)) - t.mean()) ** 2).sum()
        
#         return 1 - u / v

    def marginal_likelihood(self, x=None):
        if x is not None:
            t = self.predict(x)
        else:
            t = self.data_t

        self.inv_a = np.diag(1 / self.alpha)
        
        tmp = np.dot(self.F, np.dot(self.inv_a, self.F.T))
        e = np.eye(tmp.shape[0]) / self.betta

        u = e + tmp
        tmp = np.linalg.inv(u)
        tmp = -0.5 * np.dot(t.T, np.dot(tmp, t))
        det = logm(u).trace()
        tmp = tmp - np.log(2 * np.pi)*(self.N / 2) - det / 2
        return tmp

In [4]:
fr_2_cl_my = RVMRegression(kernel='rbf', gamma=1e-5, number_of_iterations=8000)
fr_2_cl_my.fit(train_x, train_y)
# fr_2_cl_my.score(test_x, test_y)

NameError: name 'train_x' is not defined

In [9]:
mean_squared_error(fr_2_cl_my.predict(test_x), test_y) ** 0.5

NameError: name 'test_x' is not defined

In [ ]:
plt.scatter(test_x[:,1], test_y)
plt.scatter(test_x[:,1], fr_2_cl_my.predict(test_x), c='r')
plt.show()

In [ ]:
fr_2_cl_my.F

In [ ]:
print('Friedman data set')

np.random.seed(2)
n_exp = 1
rel_vec = 0
RMSE = 0
for i in range(n_exp):
    train_x, train_y = make_friedman3(240, noise=1/3)
    fr_2_cl_em = RVMRegression(kernel="rbf",gamma=1.3975104130536649e-05, algorithm='EM', number_of_iterations=100)
    log_prob_em, rel_vec_numb_em = fr_2_cl_em.fit(train_x, train_y.reshape(-1, 1))
    rel_vec += len(fr_2_cl_em.rel_ind)

    valid_x, valid_y = make_friedman3(1000)
    pred_y = fr_2_cl_em.predict(valid_x)

    RMSE += mean_squared_error(valid_y, pred_y) ** 0.5

print("Vectors: ", rel_vec / n_exp)
print("mean square = ", RMSE / n_exp)

In [ ]:
train_pred_em = fr_2_cl_em.predict(train_x)
train_pred_tipping = fr_2_cl_em.predict(train_x)

print(mean_squared_error(train_y, train_pred_em), mean_squared_error(train_y, train_pred_tipping))

In [ ]:
print('Friedman data set')

np.random.seed(2)
n_exp = 1
rel_vec = 0
RMSE = 0
for i in range(n_exp):
    train_x, train_y = make_friedman3(240, noise=0)
    fr_2_cl_tipping = RVMRegression(kernel="rbf", gamma=1.3975104130536649e-05, number_of_iterations=200)
    log_prob_tipping, rel_vec_numb_tipping = fr_2_cl_tipping.fit(train_x, train_y.reshape(-1, 1))
    rel_vec += len(fr_2_cl_tipping.rel_ind)

    valid_x, valid_y = make_friedman3(800)
    pred_y = fr_2_cl_tipping.predict(valid_x)

    RMSE += mean_squared_error(valid_y, pred_y)

print("Vectors: ", rel_vec / n_exp)
print("RMSE = ", RMSE / n_exp)

In [ ]:
log_prob_tipping[-1]

In [ ]:
mean_squared_error(train_y, fr_2_cl_tipping.predict(train_x))

In [ ]:
valid_x, valid_y = make_friedman3(1000)
pred_y = fr_2_cl_tipping.predict(valid_x)
mean_squared_error(valid_y, pred_y)

In [ ]:
fr_2_cl_tipping.score(valid_x, valid_y)

In [ ]:
valid_x, valid_y = make_friedman3(10000)
pred_y = fr_2_cl_tipping.predict(valid_x)

mean_squared_error(valid_y, pred_y)

In [ ]:
from sklearn.svm import SVR

cl = SVR(C=4500, kernel='rbf', gamma=4.4984326689694439e-06)
cl.fit(train_x, train_y)
print('score', cl.score(valid_x, valid_y))

svm_pred = cl.predict(valid_x)
print('MSE', mean_squared_error(svm_pred, valid_y))

In [ ]:
parameters = {'kernel':['rbf'], 'gamma':np.logspace(-7, 2, 10), 
              'C':np.logspace(2, 9, 10)}
clf = GridSearchCV(SVR(), parameters)
clf.fit(train_x, train_y)

In [ ]:
clf.best_params_

In [ ]:
svm_pred = cl.predict(valid_x)
mean_squared_error(svm_pred, valid_y) ** 0.5

In [ ]:
parameters = {'kernel':['rbf'], 'gamma':np.linspace(1.3738237958832637e-05 / 2, 1.3738237958832637e-05 * 2, 30), 'algorithm':['tipping']}
clf = GridSearchCV(RVMRegression(), [parameters])
clf.fit(train_x, train_y)

In [ ]:
clf.best_params_, clf.best_score_

In [ ]:
fr_2_cl_tipping.score(valid_x, valid_y)

In [ ]:
plt.scatter(valid_x[:,2], valid_y)
plt.scatter(valid_x[:,2], svm_pred, c='r')

plt.show()

In [ ]:
from sklearn.datasets import load_boston
for i in range(n_exp):
    boston = load_boston()
    train_x = boston.data[:481]
    train_y = boston.target[:481]

    valid_x = boston.data[481:]
    valid_y = boston.target[481:]

    cl = RVMRegression(kernel="rbf", gamma=0.0001)
    print(train_x.shape, train_y.shape)
    log_prob_tipping, rel_vec_numb_tipping = cl.fit(np.matrix(train_x), np.matrix(train_y.reshape(481, 1)))
    rel_vec += len(cl.rel_ind)

    pred_y = cl.predict(valid_x)

    RMSE += mean_squared_error(valid_y, pred_y) ** 0.5

In [ ]:
for i in range(n_exp):
    boston = load_boston()
    train_x = boston.data[:481]
    train_y = boston.target[:481]

    valid_x = boston.data[481:]
    valid_y = boston.target[481:]

    cl = RVMRegression(kernel="rbf", gamma=0.0001, algo='EM', number_of_iterations=100)
    print(train_x.shape, train_y.shape)
    log_prob_em, rel_vec_numb_em = cl.fit(np.matrix(train_x), np.matrix(train_y.reshape(481, 1)))
    rel_vec += len(cl.rel_ind)

    pred_y = cl.predict(valid_x)

    RMSE += mean_squared_error(valid_y, pred_y) ** 0.5

In [ ]:
rel_vec_numb_em[-1]

In [ ]:
k, n = 1, 100
x1 = log_prob_tipping[k:n]
x2 = log_prob_em[k:n]

f, (ax1, ax2) = plt.subplots(2, sharex=True)

f.suptitle('Boston Housing data set', fontsize=14, fontweight='bold')
# plt.subplot(2, 1, 1, sharex=True)
ax1.plot(np.arange(len(x1)) + k, x1, 'g-', label="Tipping")
ax1.plot(np.arange(len(x2)) + k, x2, 'b-', label="EM")
ax1.plot((k, n), (x1[-1], x1[-1]), 'g--')

ax1.axis([k - 1, n, x1.min() - 0.5, x1.max() + 5])
ax1.set_xlabel('Iteration')
ax1.set_title('log likelihood (start at ' + str(log_prob_tipping[0]) + ')')
ax1.legend(bbox_to_anchor=(0.65, 0.88), loc=2, borderaxespad=0.)
# plt.show()

y1 = rel_vec_numb_tipping[k:n]
y2 = rel_vec_numb_em[k:n]

# plt.subplot(2, 1, 2, sharex=True)
ax2.plot(np.arange(len(y1)) + k, y1, 'g-', label='Tipping (' + str(rel_vec_numb_tipping[-1]) + ')')
ax2.plot(np.arange(len(y2)) + k, y2, 'b-', label='EM (' +  str(rel_vec_numb_em[-1]) + ')')

ax2.set_xlabel('Iteration')
ax2.set_title('Number of relevant vectors')
ax2.legend(bbox_to_anchor=(0.65, 0.8), loc=2, borderaxespad=0.)

plt.show()

In [ ]:
plt.scatter(np.arange(1, len(fr_2_cl.alpha) + 1), fr_2_cl.alpha)
plt.scatter(np.arange(1, len(fr_2_cl_em.alpha) + 1), fr_2_cl_em.alpha, c='g')
plt.axis([0, 300, -5000, 5000])
plt.show()

In [ ]:
plt.scatter(np.arange(1, len(fr_2_cl.w.A[:,0]) + 1), fr_2_cl.w.A[:,0])
plt.scatter(np.arange(1, len(fr_2_cl_em.w.A[:,0]) + 1), fr_2_cl_em.w.A[:,0], c='g')
plt.show()

In [ ]:
x = fr_2_cl.w.A[:,0]
# x = x[np.abs(x) > 1e-3]
x = np.select([(np.abs(x) > 1e-3)], [x], [100000] * len(x))
plt.scatter(np.arange(1, len(x) + 1), x, label='EM '  + str(len(x)))
plt.axis([0, len(x), -7, 7])

x = fr_2_cl_em.w.A[:,0]
# x = x[np.abs(x) > 1e-3]
x = np.select([(np.abs(x) > 1e-3)], [x], [100000] * len(x))
plt.scatter(np.arange(1, len(x) + 1), x, marker='x', c='r', label='Tipping '  + str(len(x)))

plt.xlabel('Coordinate, i')
plt.ylabel('w[i]')
plt.title('Weights')
# plt.legend(bbox_to_anchor=(0.7, 0.2), loc=2, borderaxespad=0.)

plt.show()

In [ ]:
x = fr_2_cl.alpha
# x = x[np.abs(x) < 1e200]
plt.scatter(np.arange(1, len(x) + 1), x, label='EM ')

x = fr_2_cl_em.alpha
# x = x[np.abs(x) < 1e200]
plt.scatter(np.arange(1, len(x) + 1), x, c='r', label='Tipping ' + str(len(x)))

plt.axis([0, len(x), -1e4, 1e5])
plt.xlabel('Coordinate, i')
plt.ylabel('Alpha[i]')
plt.legend(bbox_to_anchor=(0.65, 0.8), loc=2, borderaxespad=0.)
plt.grid()

plt.show()

In [ ]:
x = fr_2_cl.w.A[:,0]
plt.scatter(np.arange(1, len(x) + 1), x, marker='x', label='EM '  + str(len(x)))
plt.show()

In [ ]:
# f, (ax1, ax2) = plt.subplots(2, sharex=True)
x = fr_2_cl.w.A[:,0]
y = fr_2_cl.alpha

plt.scatter(x, y)
plt.axis([-1e-3, 1e-3, -2e4, 4e5])

# plt.yticks(np.arange(-1, 50000, 100))
# plt.xticks(np.arange(-10, 10, 1))
plt.grid()
# ax1.scatter(np.arange(len(y)) + 1, y)
# ax1.axis([0, 20, -100, 1e5])

# ax2.scatter(np.arange(len(x)) + 1, x, marker='x', c='b')
# ax2.axis([0, 20, -1e-3, 1e-3])

plt.show()

In [ ]:
f, (ax1, ax2) = plt.subplots(2)

# alpha to w
x = fr_2_cl.w.A[:,0]
y = fr_2_cl.alpha

ax1.set_title('Alpha for EM ')
ax1.scatter(x, y)
ax1.axis([-1e-3, 1e-3, -2e4, 4e5])
ax1.set_xlabel('Weight')
ax1.set_ylabel('Alpha')
ax1.grid()

# weights

x = fr_2_cl.w.A[:,0]
# x = x[np.abs(x) > 1e-3]
x = np.select([(np.abs(x) > 1e-3)], [x], [100000] * len(x))
ax2.scatter(np.arange(1, len(x) + 1), x, label='EM')
ax2.axis([0, len(x), -7, 7])

x = fr_2_cl_em.w.A[:,0]
# x = x[np.abs(x) > 1e-3]
x = np.select([(np.abs(x) > 1e-3)], [x], [100000] * len(x))
ax2.scatter(np.arange(1, len(x) + 1), x, marker='x', c='r', label='Tipping ')

ax2.set_xlabel('Coordinate, i')
ax2.set_ylabel('w[i]')
ax2.set_title('Weights')
ax2.legend(bbox_to_anchor=(0.65, 0.85), loc=2, borderaxespad=0.)
ax2.grid()

plt.show()

In [ ]:
n_exp = 1
RMSE = 0
for i in range(n_exp):

    sinc_cl = RVMRegression(kernel=linear_spline_kernel, number_of_iterations=5000, bias_used=True, calc_prob=False)
    a, b = sinc_cl.fit(train_x.reshape((-1, 1)), train_y.reshape((-1, 1)))

    pred_y = sinc_cl.predict(valid_x)
    plt.scatter(train_x, train_y, c="r", marker='x')
    
#     plt.scatter(valid_x, pred_y, c='y')
    # plt.plot(valid_x, valid_y)
    
#     plt.scatter(sinc_cl.relevance_x, sinc_cl.relevance_y, c="g")
    RMSE += mean_squared_error(valid_y, pred_y) ** 0.5
    print('RMSE', mean_squared_error(valid_y, pred_y) ** 0.5)
    
    x = np.arange(L, R, 0.2)
    y = np.sinc(x / np.pi)
    
    plt.plot(x, y, "-y")
    
    y_p = sinc_cl.predict(x.reshape((-1, 1)))
    plt.plot(x, y_p, '-b')
    
    plt.title('Root-mean-square = ' + str(mean_squared_error(valid_y, pred_y) ** 0.5) + '\n'
          + str(len(sinc_cl.w)) + ' Relevance vectors')
    
    plt.axis([-10, 10, -0.3, 1.2])
#     plt.savefig('unif_samp_sinc_' + str(int(k)) + '.png')
    plt.show()

print(RMSE / n_exp)

In [ ]:
0.02108**2

In [ ]:
sinc_cl.score(valid_x, valid_y)

In [ ]:
L = -10
R = 10
# np.random.seed(1)

train_x = np.linspace(L, R, 100)
train_y = np.sinc(train_x / np.pi)

# не забывай про seed!!!
train_y += np.random.uniform(-0.1, 0.1, len(train_y))

train_x = train_x.reshape((-1, 1))

test_x = np.linspace(L, R, 1000)
test_x = valid_x.reshape((-1, 1))
test_y = np.sinc(valid_x / np.pi).reshape((-1,))

In [ ]:
mean_squared_error(valid_y.reshape((-1,)), pred_y) ** 0.5

In [ ]:
((pred_y.reshape((-1,)) - valid_y.reshape(-1,))).max()

In [ ]:
pred_y.shape

In [ ]:
sinc_cl.score(valid_x, valid_y.reshape((-1,)))

In [ ]:
start, end = 6000, 10000
plt.plot(np.arange(end - start) + start, a[start:end])
plt.show()

In [ ]:
cl = RVR(kernel=linear_spline_kernel)
cl.fit(train_x.reshape((-1, 1)), train_y.reshape((-1,)))

In [ ]:
mean_squared_error(cl.predict(valid_x), valid_y.reshape((-1,))) ** 0.5

In [ ]:
sinc_cl.w.A[:,0][sinc_cl.rel_ind]

In [ ]:
((cl.predict(valid_x) - valid_y.reshape(-1,))).max()

In [ ]:
from sklearn.cross_validation import train_test_split

In [5]:
train_x, train_y = make_friedman3(240, noise=1/3)
test_x, test_y = make_friedman3(1000)
# train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=1-0.24, random_state=42)

In [59]:
fr_2_cl_my = RVMRegression(kernel='rbf', betta=100, alpha=1e-6, gamma=6.3095734448019296e-07)
a, b = fr_2_cl_my.fit(train_x, train_y)
fr_2_cl_my.score(test_x, test_y)

0.25386594887409308

In [60]:
mean_squared_error(fr_2_cl_my.predict(test_x), test_y)

0.067364621398883745

In [62]:
al2, w2 = fr_2_cl_my.alpha, fr_2_cl_my.w

In [64]:
w1, w2

(array([  2.79037799e-04,   3.23829824e+01,   3.22854488e-03,
          6.47866679e+04,   7.49440932e+05,   1.33219656e+05,
         -2.06793533e+04,  -5.91796080e+04,  -8.67591436e+05]),
 array([ 53.12425216,   1.18377345, -52.36903984]))

In [ ]:
fr_2_cl_rvr = RVR(kernel="rbf")
fr_2_cl_rvr.fit(train_x, train_y)
fr_2_cl_rvr.score(test_x, test_y)

In [ ]:
params = {'gamma':np.linspace(1e-5, 1e-4, 100), 'kernel':['rbf'], 'number_of_iterations':[5000]}
clf = GridSearchCV(RVMRegression(), [params])
clf.fit(train_x, train_y)

In [86]:
cv_x, cv_y = make_friedman3(n_samples=500, noise=1/3)

In [ ]:
params = {'coef1':np.logspace(-5, 0, 10), 'kernel':['rbf'], 'alpha':np.logspace(-6, -2, 10), 
          'beta': np.logspace(0, 5, 10)}
clf = GridSearchCV(RVR(), [params])
clf.fit(cv_x, cv_y)

In [84]:
clf.best_params_, clf.best_score_

({'alpha': 1.2589254117941661e-08,
  'beta': 100.0,
  'coef1': 9.9999999999999995e-07,
  'kernel': 'rbf'},
 0.21373419318724338)

In [85]:
mean_squared_error(clf.best_estimator_.predict(test_x), test_y)

0.038864520776495284

In [80]:
clf.best_estimator_.score(test_x, test_y)

-163.42304446326801

In [81]:
clf.best_estimator_.score(train_x, train_y)

-67.76118521197445

In [82]:
plt.scatter(test_x[:,1], test_y)
plt.scatter(test_x[:,1], clf.best_estimator_.predict(test_x), c='r')
plt.show()

In [ ]:
def linear_spline_kernel(x, y):    
    ans = []
    for x_n in x:
        x_n = x_n[0]
        tmp = []
        for y_n in y:
            x_m = y_n[0]
            m = min(x_n, x_m)
            tmp.append(1 + x_n * x_m + x_n * x_m * m - (x_n + x_m) * (m**2) / 2 + (m**3) / 3)
        ans.append(tmp)
    
    return np.array(ans)

In [ ]:
parameters = {'C':[10], 'gamma':np.logspace(-9, -6, 10),
             'kernel':['linear']}
clf = GridSearchCV(SVR(), [parameters])
clf.fit(train_x, train_y)

In [ ]:
clf.best_params_, clf.best_score_

In [ ]:
mean_squared_error(clf.best_estimator_.predict(test_x), test_y)

In [ ]:
clf.best_estimator_.score(test_x, test_y)

In [ ]:
sinc_svm = SVR(kernel='rbf', C=10, gamma=0.0000001)
sinc_svm.fit(train_x, train_y)

In [ ]:
mean_squared_error(sinc_svm.predict(test_x), test_y)

In [ ]:
sinc_svm.tol

In [ ]:
from sklearn.grid_search import GridSearchCV

In [ ]:
parameters = {'C':np.logspace(-4, -4, 20), 'epsilon':np.logspace(-4, 4, 20)} #, 'epsilon':np.logspace(-5, -2, 20)}
clf = GridSearchCV(MySVC(), parameters)
clf.fit(train_x.reshape((-1, 1)), train_y.reshape((-1,)))

In [ ]:
class MySVC(BaseEstimator, RegressorMixin):
    def __init__( self, C=1, epsilon=0.01):
        self.C = C
        self.epsilon = epsilon
    def fit( self, X, y ):
        kernel = linear_spline_kernel
        self.svc_ = SVR(kernel=linear_spline_kernel, C=self.C, epsilon=self.epsilon)
        return self.svc_.fit( X, y )
    def predict( self, X ):
        return self.svc_.predict( X )
                         
    def score(self, X, y):
        return self.svc_.score(X, y)

In [ ]:
MySVC().fit(train_x.reshape((-1, 1)), train_y.reshape((-1,))).score(train_x.reshape((-1, 1)), train_y.reshape((-1,)))

In [ ]:
clf.best_score_, clf.best_params_

In [ ]:
mean_squared_error(clf.best_estimator_.predict(valid_x.reshape(-1, 1)), valid_y.reshape(-1,)) ** 0.5

In [ ]:
plt.scatter(train_x, train_y)
plt.show()

In [ ]:
clf.best_estimator_.svc_.support_

In [ ]:
plt.plot(valid_x, clf.best_estimator_.predict(valid_x.reshape(-1, 1)))
plt.plot(valid_x, np.sinc(valid_x / np.pi))
plt.show()

In [ ]:
clf.best_estimator_.svc_.support_vectors_

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()

In [ ]:
clsvr = SVR(C=100)
clsvr.fit(iris.data, iris.target)

In [ ]:
clsvr.score(iris.data, iris.target)

In [ ]:
len(clf.best_estimator_.svc_.support_), len(clf.best_estimator_.svc_.support_vectors_)